In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

# Load the data into a Pandas DataFrame
df = pd.read_csv('data.csv')

# Split the data into training and validation sets
train_df = df.sample(frac=0.8, random_state=42)
val_df = df.drop(train_df.index)

# Load the BERT tokenizer and encode the text data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_text(text):
    return tokenizer.encode_plus(text, add_special_tokens=True, 
                                  max_length=64, padding='max_length', 
                                  return_attention_mask=True, return_tensors='tf')

train_ds = tf.data.Dataset.from_tensor_slices(
    (dict(tokenize_text(text)), label) for text, label in zip(train_df['text'], train_df['label'])
).batch(32)

val_ds = tf.data.Dataset.from_tensor_slices(
    (dict(tokenize_text(text)), label) for text, label in zip(val_df['text'], val_df['label'])
).batch(32)

# Load the pre-trained BERT model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

# Compile the model with appropriate loss and metrics
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

# Train the model
model.fit(train_ds, epochs=5, validation_data=val_ds)
